#Slot Prediction for Localities
The choice of delivery time slots has to balance marketing and  operational considerations  resulting in cost savings for the organization. Identifying specific time slots for each localities when there is higher chance of success is key to success. Also, if there is any distinction can be made between residential and commercial localities

In [4]:
#Importing Libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import generic_utils
import config
import pandas as pd


In [6]:
#Reading the Locality Data for last 3 months

#Date Range - 23rd Oct to 24th Jan 2019

timeSlotData= pd.read_csv("clustering_data.csv")




In [9]:
# Filtering for only the address column 

addressField = timeSlotData[["combined_address"]]

In [41]:
addressField.head()

path_to_csv='/data/Projects/Amir/alias_size_1.csv'

In [147]:
import config
import generic_utils

alias_size_1= pd.read_csv('alias_size_1.csv')
alias_size_2= pd.read_csv('alias_size_2.csv')
alias_size_3= pd.read_csv('alias_size_3.csv')
phrase_alias = pd.read_csv("phrase_alias")

In [139]:
def replace_alias(address):
        """Hard coded alias are stored for some commonly occurring incorrect 
        spellings people wirite, for ex. bldg -> building, rd no -> road 
        number etc. Given an address, if there are any incorrect spelling 
        present in the address this function replaces them with their correct 
        alias.

        Higher length incorrect spellings are replaced first and then the 
        lower length incorrect spellings are replcaed. For example we first 
        replace length 3 incorrect spellings, then length 2 and then length 1.

        :param str address: Given address whose incorrect spellings are to be 
                replaced

        :returns: Refined address with incorrect spellings replaced with 
                corresponding aliases

        """
        splitted_address = address.split(',')
        replaced_address = []
        for phrase in splitted_address:
            words = phrase.split()
            num_word = len(words)
            # FIXME: Need to make the algorithm efficient using Trie structure
            for j, word in enumerate(words):
                size_3_replaced = False
                size_2_replaced = False
                # If starting from this word length 3 size incorrect spelling
                # exist then replace it with correspondig alias, else look
                # for length 2 incorrect spelling and then length 1 incorrect
                # spelling
                if j + 2 < num_word:
                    new_word = alias_size_3.get(
                        word + " " + words[j + 1] + " " + words[j + 2])
                    if new_word is not None:
                        words[j] = new_word
                        del words[j + 2]
                        del words[j + 1]
                        num_word = len(words)
                        size_3_replaced = True
                if not size_3_replaced:
                    if j + 1 < num_word:
                        new_word = alias_size_2.get(
                            word + " " + words[j + 1])
                        if new_word is not None:
                            words[j] = new_word
                            del words[j + 1]
                            num_word = len(words)
                            size_2_replaced = True
                    if not size_2_replaced:
                        words[j] = alias_size_1.get(word, word)
            replaced_phrase = " ".join(words)
            if replaced_phrase and not replaced_phrase.isspace():
                replaced_phrase = " ".join(replaced_phrase.split())
                replaced_address.append(replaced_phrase)
        return ",".join(replaced_address)


In [144]:
def _clean_at_phrase_level(address):
        """Clean the address at comma separated phrase level.

        :param str address: Address to be cleaned at phrase level.

        """
        splitted_address = address.split(',')
        address_array = []
        for phrase in splitted_address:
            if (phrase and not phrase.isspace()):
                phrase = " ".join(phrase.split())
                clean_phrase = phrase_alias.get(phrase, phrase)
                numeric_match = re.match(
                    r"^([0-9]+)\s?(st|nd|rd|th)$", clean_phrase)
                if numeric_match:
                    clean_phrase = numeric_match.group(1)
                    pattern = (r"^(([0-9]+|[a-z]{1,2}) floor|"
                            r"floor( (no|num|number))? ([0-9]+(st|nd|rd|th)?|"
                            r"[a-z]{1,2}))$")
                    floor_match = re.match(pattern, clean_phrase)
                    clean_phrase = '' if floor_match else clean_phrase
                if clean_phrase:
                    address_array.append(clean_phrase)
        seen = set()
        seen_add = seen.add
        address_array_unique = [x for x in reversed(
            address_array) if not (x in seen or seen_add(x))]
        address_array_unique = list(reversed(address_array_unique))
        return ",".join(address_array_unique)


In [152]:
city='Gurugram'
state='Haryana'

def clean_city_and_state(city, state):
        """Clean the city and state.

        :param str city: City
        :param str state: State

        :returns: City & State after cleaning

        """
        city_and_state = city + state
        if city_and_state.count('/') == 1:
            city, state = city_and_state.split('/')
        city = clean_address_new(city)
        state = clean_address_new(state)
        # Many time merchant writes 'in-' before state name. i.e. in-punjab
        if state.startswith('in '):
            state = state[3:]
        state = self.state_abbreviations.get(state, state)
        return city, state



In [ ]:
 def remove_intermediate_city_state_pincode(address_data):
        """Given the address data, if city/state field is non-empty then 
        remove the city/state name from in-between the address. City/state 
        can be appended at the end of address later on whenever needed. This
        function removes the pincode from the address.


        """
        address = address_data['address']
        city = address_data['city']
        state = address_data['state']
        pincode = address_data['pincode']
        modified_phrases = []
        # if pincode is present in both address field and pincode field,
        # update with the rightmost one present in address field.
        pincode_match = re.findall(r"\b([0-9]{6})\b", address)
        pincode_in_add = pincode_match[-1] if pincode_match else None
        pin = pincode_in_add if pincode_in_add else pincode
        address_data['pincode'] = pin
        all_pincodes_set = set([pincode])
        if pincode_match:
            all_pincodes_set |= set(pincode_match)
        for phrase in address.split(','):
            # Keep all phrases of address other than pincode, city and state
            # FIXME: Check if using levenshtein distance below can be harmful
            # and should be removed OR should it be replaced with fuzzy
            # FIXME: chack if combination of city & state (i.e. 'gurgaon
            # haryana') should also be removed
            if not ((phrase in all_pincodes_set) or
                    (city and l_distance(phrase, city) < 2) or
                    (state and l_distance(phrase, state) < 2)):
                # Note that pincode/city/state are removed only if they
                # represent some independent phrase of the address and not
                # otherwise.
                modified_phrases.append(phrase)
        if self.remove_city_state_token_from_address and modified_phrases:
            tokens_last_phrase = modified_phrases[-1].split()
            if tokens_last_phrase and tokens_last_phrase[-1] in (city, state):
                modified_phrases[-1] = ' '.join(tokens_last_phrase[:-1])
        address_data['address'] = ','.join(modified_phrases)


In [185]:
def clean_address_new(address):
    address = address.replace("/n", " ")
    # Addresses from some merchants contain noisy words/codes like: &#21,
    # &#45 etc.
    address = re.sub(r"&#\w+(?=\b)", " ", address)
    address = address.replace("&", " and ")
    address = address.replace(";", ",")
    address = address.replace("'", "")
    # 'Bandra (West)' is better than 'Bandra(West)'. For such phrases
    # add space before (
    address = address.replace("(", " (")
    # Remove all useless characters
    address = re.sub(r"[^A-Za-z0-9(),]", " ", address)
    # Add space where Capital alphabet is followed by small letter i.e.
    # 'RohiniDelhi' -> 'Rohini Delhi'
    address = re.sub(r"(?<=[a-z])(?=[A-Z])", " ", address)
    address = address.lower() 
    address = segregate_joint_alphabet_numerics(address)
    address = re.sub(
        r"(?<=\b)(sec|sect)\s(?=[0-9]+\b)", "sector ", address)
    # Remove phrases for any nth floor as floor number doesn't give any
    # additional information for geocoding
    address = re.sub(r"(([0-9]+\s?)|([a-z]+))(st|nd|rd|th)\sfloor(?=\b)", ", ", address)
    address = re.sub(r"(?<=[a-z])\s+(?=pur\b)", "", address)
    # From phrases like 'ho no 18', 'plot 88' etc retain the numeric part
    # and remove rest of the part
    pattern = (r"(?<=\b)(house|ho|plot|room|shop|khasra|contact|phone|"
               r"mobile|mob)\s+((n|no|num|number)\s+)?(?=[0-9]+)")
    address = re.sub(pattern, ",", address)
    # For handling special cases like 'sector h 7', we do not remove the
    # 'h' from the address. If 'sector h no 7', we return 'sector ,7'
    # only as output.
    address = re.sub(r"(?<=\b)h\s+(n|no|num|number)\s+(?=[0-9]+)", ",", address)
    # From phrases like 'flat no b 203', 'flat d 46' etc. remove 'flat',
    # 'flat no' etc. keywords
    pattern = (r"(?<=\b)flat\s+((no|num|number)\s+)?"
               r"(?=([a-z]{1,2}|[0-9]+)\b)(?!no\b)")
    address = re.sub(pattern, ",", address)
    # Replace rd to road if there is some alphabet name of road before rd.
    # Remplacement doesn't happen in cases like 'head office, RD School'
    address = re.sub(r"(?<=[a-z]\s)\s*rd(?=\b)", " road ", address)
    address =  align_numeric_part(address)
    address =  replace_alias(address)

    # Comma before pincode and after pincode
    address = re.sub(r"\s(?=[0-9]{6}\b)", ", ", address)
    address = re.sub(r"(?<=\b[0-9]{6})\s", ", ", address)
    # Delete phone numbers
    address = re.sub(r"(?<=\b)(91\s)?[0-9]{10,12}(?=\b)", ", ", address)
    # FIXME: Instead of hardcoding country code of India in regex above,
    # make it generic for any country
    address = _clean_at_phrase_level(address)
    
    address = re.sub(r"(?<=\b)[0-9]{6}(?=\b)", ",", address)
    address = ",".join([" ".join(item.split()) for item in address.split(
            ',') if (item and not item.isspace())])
    
    address= re.sub(r"sector 44","", address)
    address= re.sub(r"haryana","", address)
    address= re.sub(r"gurugram","",address)
    address= re.sub(r"gurgaon","",address)
    address= re.sub(r"gurgam","",address)
    address= re.sub(r"gurdgaon","",address)
    address=re.sub(r","," ",address)
    adress= address.strip()
    address=re.sub(' +', ' ', address)   
    return(address)


In [186]:
addressField['Clean_Address'] = addressField['combined_address'].apply(clean_address_new)

/home/ubuntu/.venv/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [209]:
re.findall(r'\b\d+\b', 'he33llo 42 I\'m a 32 string 30')

['42', '32', '30']

In [213]:
def extract_numeric_part(address):
    numeric_part= [int(s) for s in re.findall(r'\b\d+\b',address)]
    return(numeric_part)

def remove_numeric_part(address):
    address=''.join(i for i in address if not i.isdigit())
    return(address)
    

In [214]:
addressField['Numeric_Part'] =addressField['Clean_Address'].apply(extract_numeric_part)
addressField['Clean_Address'] = addressField['Clean_Address'].apply(remove_numeric_part)

/home/ubuntu/.venv/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/ubuntu/.venv/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [216]:
#The Numbers and strings part are removed from the addresses
#The idea is to assign the 



In [188]:
addressField.to_csv("Modified_Address.csv")